In [1]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("NOMIC_API_KEY")
local_llm = "qwen2:latest"


In [2]:
### Retrieval Grader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings  

finreport_dir = "./FinRep"

# 加载本地 txt 文件
loader = DirectoryLoader(finreport_dir, glob="report.txt", loader_cls=TextLoader)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)

# 使用本地 Ollama 的 nomic-embed-text:latest 模型
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embedding_model,
)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
### Retrieval Grader

from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "药明生物成立时间"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
#print(doc_txt)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'no'}


In [4]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/home/wangke/anaconda3/envs/langgraph/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


关于药明生物的成立时间，提供的上下文中并未提及。因此，我无法提供确切的答案。


In [5]:
### Hallucination Grader

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n 
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'no'}

In [6]:
### Answer Grader

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
    Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question}
    Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

In [7]:
### Question Re-writer

# LLM
llm = ChatOllama(model=local_llm, temperature=0)

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'When was WuXi Biologics founded?'

In [8]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [9]:
### Nodes


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    # print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    # print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    # print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            # print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            # print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    # print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    # print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        # print(
        #     "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        # )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        # print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    # print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        # print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        # print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            # print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            # print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        # print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [10]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [11]:
from pprint import pprint

# Run
inputs = {"question": "药明生物未来发展前景如何，全球市场份额会如何变化？"}
for output in app.stream(inputs,{"recursion_limit": 25}):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

"Node 'retrieve':"
'\n---\n'
"Node 'grade_documents':"
'\n---\n'
"Node 'transform_query':"
'\n---\n'
"Node 'retrieve':"
'\n---\n'
"Node 'grade_documents':"
'\n---\n'
"Node 'generate':"
'\n---\n'
"Node 'transform_query':"
'\n---\n'
"Node 'retrieve':"
'\n---\n'
"Node 'grade_documents':"
'\n---\n'
"Node 'generate':"
'\n---\n'
('The document provided does not contain specific information about the '
 'projected future growth metrics or the anticipated evolution of global '
 "market share for Wuxi Biologics. It focuses on the company's capabilities "
 'and achievements, mentioning that by mid-2019, they had initiated 15 WuXiUP '
 'projects with WuXiUP continuous cell culture expression levels reaching up '
 'to 30-50g/L, which is ten times higher than traditional technologies.')


In [12]:
import os
import glob

# 假设app已经定义并可以调用
# 例如：from your_app import app

def process_query(input_dir, output_dir):
    """
    读取指定目录下的问题文件，处理问题，并写入答案到对应的文件中。

    :param input_dir: 输入目录的路径，其中包含问题文件。
    :param output_dir: 输出目录的路径，答案文件将写入此目录。
    """
    # 确保输出目录存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 使用glob模块找到目录中所有的问题文件
    query_files = glob.glob(os.path.join(input_dir, 'query*.txt'))

    # 遍历问题文件
    for i in range(0,150):  # 遍历query000.txt到query149.txt
        query_file = os.path.join(input_dir, f"query{i:03d}.txt")
        answer_file = os.path.join(output_dir, f"answer{i:03d}.txt")

        if not os.path.exists(query_file):
            print(f"文件 {query_file} 不存在，跳过处理。")
            continue

        # 读取问题文件内容
        with open(query_file, "r", encoding="utf-8") as f:
            content = f.read()
            lines = content.splitlines()

        # 提取问题1
        question = None
        for line in lines:
            if line.startswith("问题3："):
                question = line.replace("问题3：", "").strip()
                break

        if not question:
            print(f"文件 {query_file} 中未找到问题3，跳过处理。")
            continue

        # 调用app.stream获取答案
        inputs = {"question": question}
        answer = "抱歉，我无法回答您的问题。"
        try:
            for output in app.stream(inputs,{"recursion_limit": 25}):
                for key, value in output.items():
                    continue
            answer = value["generation"]
        except Exception as e:
            print(f"处理问题 {query_file} : {question}时发生错误")

        # 将问题和答案写入答案文件
        with open(answer_file, "w", encoding="utf-8") as f:
            f.write(f"问题：{question}\n")
            f.write(f"答案：{answer}\n")

        print(f"答案已写入文件 {answer_file}")

# 指定输入目录和输出文件
input_directory = './FinRep/fin_queries'
output_directory = './FinRep/SelfRAG/open'

# 调用函数执行处理操作
process_query(input_directory, output_directory)

答案已写入文件 ./FinRep/SelfRAG/open/answer000.txt
处理问题 ./FinRep/fin_queries/query001.txt : 雅生活服务的人才培养体制有什么特点，如何增强企业凝聚力？时发生错误
答案已写入文件 ./FinRep/SelfRAG/open/answer001.txt
答案已写入文件 ./FinRep/SelfRAG/open/answer002.txt
答案已写入文件 ./FinRep/SelfRAG/open/answer003.txt
处理问题 ./FinRep/fin_queries/query004.txt : 考虑到华虹半导体面临的各种挑战和风险，包括结构性问题、毛利率下滑以及竞争加剧，您如何看待公司未来的发展前景？时发生错误
答案已写入文件 ./FinRep/SelfRAG/open/answer004.txt
答案已写入文件 ./FinRep/SelfRAG/open/answer005.txt
处理问题 ./FinRep/fin_queries/query006.txt : 文章中提到的新能源汽车、5G基站和TWS耳机对于钴和碳酸锂市场的影响综合预测是什么？时发生错误
答案已写入文件 ./FinRep/SelfRAG/open/answer006.txt
答案已写入文件 ./FinRep/SelfRAG/open/answer007.txt
处理问题 ./FinRep/fin_queries/query008.txt : 2020-2022年锂供需结构会如何变化？时发生错误
答案已写入文件 ./FinRep/SelfRAG/open/answer008.txt
答案已写入文件 ./FinRep/SelfRAG/open/answer009.txt
处理问题 ./FinRep/fin_queries/query010.txt : 日本的装配式住宅认定标准与中国有何不同？时发生错误
答案已写入文件 ./FinRep/SelfRAG/open/answer010.txt
处理问题 ./FinRep/fin_queries/query011.txt : 如何看待中国财险在当前市场环境下的投资价值，考虑到疫情的影响、公司估值和行业发展趋势？时发生错误
答案已写入文件 ./FinRep/SelfRAG/o